In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn import manifold,datasets, linear_model,gaussian_process,svm,metrics,manifold,preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV, KFold, cross_validate
from sklearn.metrics import make_scorer
import warnings 
warnings.filterwarnings('ignore')
import cv2
import math 
plt.style.use('seaborn')

import scipy.io as sio

import gc

pd.options.display.max_rows = 1000
pd.set_option('display.max_columns', None)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

### 1. 数据清洗

In [14]:
# 计算每个变量的缺失率
def missing_cal(df):
    missing_series = df.isnull().sum()/df.shape[0]
    missing_df = pd.DataFrame(missing_series).reset_index()
    missing_df = missing_df.rename(columns={'index':'col',
                                            0:'missing_pct'})
    missing_df = missing_df.sort_values('missing_pct',ascending=False).reset_index(drop=True)
    return missing_df

# 缺失值剔除（单个样本）
def delete_missing_sample(df,threshold = 5):
    df2 = df.copy()
    missing_series =  df.isnull().sum(axis = 1)
    missing_list = list(missing_series)
    missing_index_list = []
    for i,j in enumerate(missing_list):
        if j>=threshold:
            missing_index_list.append(i)
    df2 = df2[~(df2.index.isin(missing_index_list))]
    print('已删除缺失变量个数在{}以上的用户{}个'.format(threshold,len(missing_index_list)))
    return df2

# 缺失值填充
def fill_missing_value(df,col_list):
    df2 = df.copy()
    for col in col_list:
        if df[col].dtypes=='object':
             df2[col] = df2[col].fillna('unknown')
        else:
            df2[col+'_meadian'] = df2[col].fillna(df2[col].median())    #中位数填充
            df2[col+'_mean'] = df2[col].fillna(df2[col].mean()) #均值填充
            df2 = df2.drop([col],axis = 1)
    return df2

# 去除常量值
def const_delete(df,col_list,threshold=0.01):
    df2 = df.copy()
    const_col = []
    rescol = []
    for col in col_list:
        const_pct = df2[col].value_counts().iloc[0]/df2[df2[col].notnull()].shape[0]
        if const_pct>=threshold:
            const_col.append(col)
            rescol.append({col:const_pct})
    df2 = df2.drop(const_col,axis=1)
    print('常变量/同值化处理的变量个数为{}'.format(len(const_col)))
    print('分别为{}'.format(rescol))
    return df2

In [15]:
wine_data = pd.read_csv('./datasets/商品价格预测.csv').reset_index(drop = True)
print(wine_data.shape)
wine_data = wine_data.replace('N.V.', np.nan).replace('NA',np.nan)
wine_data['year'] = wine_data['year'].astype('float')

# 删除缺失过于严重的样本
wine_data=delete_missing_sample(wine_data,4)
wine_data=fill_missing_value(wine_data,missing_cal(wine_data).col[0:4].values)

# 常量值处理
wine_data=const_delete(wine_data,wine_data.columns.values,1)

(7500, 11)
已删除缺失变量个数在4以上的用户7个
常变量/同值化处理的变量个数为1
分别为[{'country': 1.0}]


### 2. 特征转换与选择

In [16]:
wine_data_2 = wine_data

one_hot_fea = []
dim_red_fea_1 = []
dim_red_fea_2 = []

for col in ['winery', 'wine', 'region', 'type']:
    # 首先用one-hot编码
    le = OneHotEncoder()
    le.fit(wine_data_2[col][:,np.newaxis])
    temp_data = le.transform(wine_data_2[col][:,np.newaxis]).toarray()
    if col == 'winery':
        one_hot_fea = temp_data
    else:
        one_hot_fea = np.concatenate([one_hot_fea,temp_data],axis = 1)
    
    # 由于类别数太多，导致one-hot编码得到的特征维数太高，这容易造成过拟合问题，因此在此基础上使用PCA降维
    ndim = np.min([30,temp_data.shape[1]-1])
    svd = TruncatedSVD(n_components=ndim)
    svd.fit(temp_data)
    temp_data_2 = pd.DataFrame(svd.transform(temp_data),columns = [col+'_'+str(i) for i in range(ndim)]).reset_index(drop = True)
    if col == 'winery':
        dim_red_fea_1 = temp_data_2
    else:
        dim_red_fea_1 = np.concatenate([dim_red_fea_1,temp_data_2],axis = 1)

    ndim = np.min([50,temp_data.shape[1]-1])
    svd = TruncatedSVD(n_components=ndim)
    svd.fit(temp_data)
    temp_data_3 = pd.DataFrame(svd.transform(temp_data),columns = [col+'_'+str(i) for i in range(ndim)]).reset_index(drop = True)
    if col == 'winery':
        dim_red_fea_2 = temp_data_3
    else:
        dim_red_fea_2 = np.concatenate([dim_red_fea_2,temp_data_3],axis = 1)
    
    # wine_data_2 = wine_data_2.join(temp_data_2)
wine_data_2.drop(['winery', 'wine', 'region', 'type'],axis = 1,inplace = True)

In [17]:
X_baseline = wine_data_2[['rating','num_reviews']].values
Y = wine_data_2['price'].values

# 下面二选一
X_aug_median = wine_data_2[['body_meadian','acidity_meadian','year_meadian']].values
X_aug_mean = wine_data_2[['body_mean','acidity_mean','year_mean']].values

# 下面三选一
X_onehot = one_hot_fea
X_cat_dim1 = dim_red_fea_1
X_cat_dim2 = dim_red_fea_2

sio.savemat('./datasets/candidate_data.mat',{'X_baseline':X_baseline,'Y':Y, 'X_aug_median':X_aug_median, 'X_aug_mean':X_aug_mean, 'X_onehot':X_onehot, 'X_cat_dim1':X_cat_dim1, 'X_cat_dim2':X_cat_dim2})

### 3. 添加价格标签

In [18]:
wine_data_3 = wine_data_2
temp_data = np.zeros([wine_data_3.shape[0],1])
temp_data[wine_data_3.price>200] = 1
temp_data = pd.DataFrame(temp_data,columns = ['price_tag']).reindex()
wine_data_3 = wine_data_3.join(temp_data)

X = wine_data_3.drop(['price','price_tag'],axis = 1).values
Y = wine_data_3['price'].values

sio.savemat('./datasets/ori_data.mat',{'X':X,'Y':Y})

In [20]:
data = sio.loadmat('./datasets/ori_data.mat')

data['X'].shape

(7493, 1419)

### 4. 特征归一化

In [22]:
# 读取上一步的结果
wine_data_4 = sio.loadmat('./datasets/ori_data.mat')
X = wine_data_4['X']
Y = wine_data_4['Y']
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)
sio.savemat('./datasets/data_normalized.mat',{'X':X,'Y':Y})